In [6]:
import argparse
import traceback
import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_info
import models
import tasks
import utils.callbacks
import utils.data
import utils.email
import utils.logging
import numpy as np

In [7]:
parser = argparse.ArgumentParser()
parser = pl.Trainer.add_argparse_args(parser)
parser.add_argument(
        "--data", type=str, help="The name of the dataset", choices=("shenzhen", "losloop"), default="losloop"
    )

_StoreAction(option_strings=['--data'], dest='data', nargs=None, const=None, default='losloop', type=<class 'str'>, choices=('shenzhen', 'losloop'), help='The name of the dataset', metavar=None)

In [153]:
import scipy.io
# data = scipy.io.loadmat('data/data_detective/flu_datasets/2011-2012_flu_normalized.mat')
# data = scipy.io.loadmat('data/data_detective/civil_datasets/2013-2014_argentina.mat')
# Multi-class data
# data = scipy.io.loadmat('data/data_detective/civil_datasets/2013-2014_brazil.mat')
# data = scipy.io.loadmat('data/data_detective/civil_datasets/2013-2014_colombia.mat')
# data = scipy.io.loadmat('data/data_detective/civil_datasets/2013-2014_mexico.mat')
# data = scipy.io.loadmat('data/data_detective/civil_datasets/2013-2014_paraguay.mat')
data = scipy.io.loadmat('data/data_detective/air_datasets/china_air.mat')

In [154]:
data['X'].shape

(10, 365, 1344)

In [155]:
adj_name = 'State_adj'
y_key = 'Y5'
x_data = np.array(data['X'],dtype=np.float16)  # shape (26,550,1289) (Task ID (Spatial location), Sample (Temporal-Days), Features(TF-IDF)) # Civil Unrest Data
y_data = np.array(data[y_key],dtype=np.float16)  # shape (26, 550) # Y5
adj_data = np.array(data[adj_name],dtype=np.float16)  # shape (26, 26)

In [156]:
import pandas as pd
DF = pd.DataFrame(adj_data)
DF.to_csv("data/china_air_adj.csv", index=False, header=False)

In [157]:
y_data_3d = np.expand_dims(y_data, axis=2)
data_concat = np.concatenate((x_data, y_data_3d), axis=2)

In [158]:
y_data_3d.shape
y_data.shape

(10, 365)

In [159]:
550/10

55.0

In [160]:
data_concat.shape
data_concat.reshape(data_concat.shape[0],-1).shape

(10, 490925)

In [161]:
DF_feat = pd.DataFrame(np.swapaxes(data_concat.reshape(data_concat.shape[0],-1), 0, 1))
# DF_feat = pd.DataFrame(np.swapaxes(y_data, 0, 1))
DF_feat.head()

,0,1,2,3,4,5,6,7,8,9
0,0.134155,0.129639,0.079651,0.062927,0.145874,0.127075,0.098511,0.124207,0.172119,0.044678
1,0.149170,0.128784,0.083679,0.060272,0.140259,0.122192,0.104736,0.127075,0.177002,0.060272
2,0.168945,0.116028,0.083374,0.057037,0.147583,0.066040,0.107605,0.126953,0.140015,0.072754
3,0.176147,0.099609,0.098389,0.059082,0.145874,0.068176,0.101990,0.142212,0.125732,0.057678
4,0.204468,0.082581,0.173828,0.057281,0.135498,0.056732,0.095520,0.140259,0.132935,0.061005


In [162]:
DF_feat.to_csv("data/china_air_feat.csv", index=False, header=True)

In [151]:
# dtype=np.float16
np.unique(y_data-1)


array([0., 1., 2., 3., 4.], dtype=float16)

In [163]:
DF_Y = pd.DataFrame(np.swapaxes(y_data-1, 0, 1))
DF_Y.to_csv("data/china_air_y.csv", index=False, header=True)

In [79]:
tsss = [1, 2, 3, 4, 5, 6]
tsss[0+3:0+3+1]

[4]

In [83]:
seq_len = 2
pre_len = 1
xxx=[]
for i in range(0, (len(tsss)), (seq_len + pre_len)):
    print(tsss[i : i + seq_len])
    print(tsss[i + seq_len : i + seq_len + pre_len])
#     xxx.append(i)
    pass

[1, 2]
[3]
[4, 5]
[6]


In [46]:
len(xxx)/2

364.0

In [76]:
342774+1251

344025

In [85]:
688050/1251

550.0